In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Perfrom Pixelwise SSIM for Anamoly and Noraml regions Separately

In [ ]:
## Load all the necessary packages

%matplotlib inline

import matplotlib.pyplot as plt
import skimage.transform
from skimage import data, io, filters
import numpy as np
from numpy import array
from skimage.transform import rescale, resize
from skimage.transform import resize
import os
import PIL
import pandas as pd
import imageio
import tensorflow as tf
import math


from matplotlib.pyplot import imread
import h5py
import cv2

In [ ]:
## Load Directory Paths

def load_path(path):
    directories = []
    if os.path.isdir(path):
        print('directory path: ', path)
        directories.append(path)
    for elem in os.listdir(path): #check for nested dir within parent dir
        if os.path.isdir(os.path.join(path,elem)):
            print('inside nested dir')
            directories = directories + load_path(os.path.join(path,elem))
            directories.append(os.path.join(path,elem))
    print('directories: ', directories)
    return directories

In [ ]:
## Load Images from directory

def load_data_from_dirs(dirs, ext):
    files = []
    file_names = []
    count = 0
    
    for d in dirs:
        for f in os.listdir(d):
            if f.endswith(ext):
                image = h5py.File(os.path.join(d,f), 'r')
                files.append(image)
                file_names.append(os.path.join(d,f))
                count = count + 1
    
    print('list of files: ',len(files), 'file: ', files[0])
    print('list of files path: ', len(file_names), 'file path: ', file_names[0])
    print('Files Read: ',count)
    
    return files   

In [ ]:
## Load Images

def load_data(directory, ext):
    files = load_data_from_dirs(load_path(directory), ext)
    return files

In [ ]:
files = load_data("../input/mydataset/BrainTumorData/", ".mat")

## Analyzing Attributes of .mat file using HDF5

In [ ]:
print(files[0].keys())
print(files[0]['cjdata'])
print(files[0]['cjdata']['PID'])
print(files[0]['cjdata']['image'])
print(files[0]['cjdata']['label'])
print(files[0]['cjdata']['tumorBorder'])
print(files[0]['cjdata']['tumorMask'])

## Attributes and their meaning

### Here we can clearly see that image has following attributes:

1. cjdata.label: 1 for meningioma, 2 for glioma, 3 for pituitary tumor
2. cjdata.PID: patient ID
3. cjdata.image: image data
4. cjdata.tumorBorder: a vector storing the coordinates of discrete points on the tumour border. For example, [x1, y1, x2, y2,…] in which x1, y1 are planar coordinates on tumour border. It was generated by manually delineating the tumour border. So we can use it to generate binary image of tumour mask.
5. cjdata.tumorMask: a binary image with 1s indicating tumour region

## Let's print each attribute one by one

In [ ]:
## cjdata.PID: patient ID

print('Patient ID: ', files[0]['cjdata']['PID'][()])

In [ ]:
## cjdata.image: image data

print('Tumour Image Array: ', files[0]['cjdata']['image'][()])

In [ ]:
plt.imshow(files[0]['cjdata']['image'][()].squeeze(), cmap='gray')

## Let's analyze the image attributes

In [ ]:
img = PIL.Image.fromarray(files[0]['cjdata']['image'][()])
print(img.format)
print(img.mode)
print(img.size)
print(img.palette)
print(img.info)

In [ ]:
## cjdata.label: 1 for meningioma, 2 for glioma, 3 for pituitary tumor

print('label: ', files[0]['cjdata']['label'][()])

In [ ]:
## cjdata.tumorBorder: a vector storing the coordinates of discrete points on the tumour border. 
## For example, [x1, y1, x2, y2,…] in which x1, y1 are planar coordinates on tumour border. 
## It was generated by manually delineating the tumour border. So we can use it to generate binary image of tumour mask.

print('Tumour Border: ', files[0]['cjdata']['tumorBorder'][()])

In [ ]:
## cjdata.tumorMask: a binary image with 1s indicating tumour region

print('Tumour Mask: ',files[0]['cjdata']['tumorMask'][()])

## Let's visualize the tumor mask

In [ ]:
plt.imshow(files[0]['cjdata']['tumorMask'][()].squeeze(), cmap='gray')
plt.axis('off')

## Let's extract the images and their respective masks

In [ ]:
## Lets extract 1000 images 

images_h5 = files[:1000]

In [ ]:
image_arr = [array(img_h5['cjdata']['image'][()], dtype=np.float64) for img_h5 in images_h5]

In [ ]:
mask_arr = [array(img_h5['cjdata']['tumorMask'][()],dtype=np.float64) for img_h5 in images_h5]

## Let's see if there are any invalid shape images. Since we only want to process 512x512 images

In [ ]:
# checking for invalid shape images

for idx, img in enumerate(image_arr):
    if img.shape != (512,512):
        print(idx, img.shape)

In [ ]:
# checking for invalid shape tumor masks

for idx, img in enumerate(mask_arr):
    if img.shape != (512,512):
        print(idx, img.shape)

## Lets create a function to display 10 images in our dataset

In [ ]:
def displayImages(img_arr):    
    width=8
    height=8
    rows = 3
    cols = 4
    axes=[]

    fig=plt.figure(figsize=(10,10))

    for i in range(rows * cols):
        axes.append( fig.add_subplot(rows, cols, i+1) )
        subplot_title=("Image: "+str(i))
        axes[-1].set_title(subplot_title)  
        plt.imshow(np.asarray(img_arr[i], dtype=float).squeeze(), cmap='gray')
        plt.axis('off')
    fig.tight_layout()    
    plt.show()

In [ ]:
## Display intial 10 images in our image array

displayImages(image_arr)

### Let's get Tumour Region with the help of mask

In [ ]:
## To each image we apply the mask and store the result in the tumor array

tumour_arr = [img * mask for img, mask in zip(image_arr, mask_arr)]

## Now we will display the tumor region

In [ ]:
displayImages(tumour_arr)

In [ ]:
tmr = tumour_arr[0]
tmr = tmr.flatten()
non_zeros = list(filter(lambda x: x != 0, tmr))
print(len(tmr), len(non_zeros))

## Now we will create a gaussian filter

In [ ]:
def gaussian(window_size, sigma):
    
    """
    Generates a list of Tensor values drawn from a gaussian distribution with standard
    diviation = sigma and sum of all elements = 1.

    Length of list = window_size
    """    
    
    gauss =  tf.convert_to_tensor([math.exp(-(x - window_size//2)**2/float(2*sigma**2)) for x in range(window_size)])
    
    return gauss/tf.reduce_sum(gauss)

In [ ]:
gauss_dis = gaussian(11, 1.5)
print("Distribution: ", gauss_dis)
print("Sum of Gauss Distribution:", np.sum(gauss_dis))

In [ ]:
def ssim(org, gen):
    
    im1 = np.array(list(filter(lambda x: x != 0, org.flatten())))
    im2 = np.array(list(filter(lambda x: x != 0, gen.flatten())))
    win_size = len(im1)
    
    kernel = gaussian(win_size, 1.5)
    
    # calculating the mu parameter (locally) for both images using a gaussian filter 
    # calculates the luminosity params
    
    mu1 = np.convolve(im1, kernel)
    mu2 = np.convolve(im2, kernel)
    
    mu1_sq = mu1 ** 2
    mu2_sq = mu2 ** 2 
    mu12 = mu1 * mu2

    # now we calculate the sigma square parameter
    # Sigma deals with the contrast component 
    sigma1_sq = np.convolve(im1 * im1, kernel) - mu1_sq
    sigma2_sq = np.convolve(im2 * im2, kernel) - mu2_sq
    sigma12 =  np.convolve(im1 * im2, kernel) - mu12

    # Some constants for stability 
    C1 = (0.01 ) ** 2 
    C2 = (0.03 ) ** 2 

    contrast_metric = (2.0 * sigma12 + C2) / (sigma1_sq + sigma2_sq + C2)
    contrast_metric = tf.reduce_mean(contrast_metric)

    numerator1 = 2 * mu12 + C1  
    numerator2 = 2 * sigma12 + C2
    denominator1 = mu1_sq + mu2_sq + C1 
    denominator2 = sigma1_sq + sigma2_sq + C2

    ssim_score = (numerator1 * numerator2) / (denominator1 * denominator2)
    
    return ssim_score.mean()

In [ ]:
ssim(tumour_arr[1], tumour_arr[1])